This program allows the user to select their own text document which should contain a list of unbalanced chemical equations. The program balances all of the equations, and allows the user to do stoichiometry calculations (either in grams or in moles). An example text document containing unbalanced chemical equations will look like:

CO2 + H2O = C6H12O6 + O2  
Al + HCl = AlCl3 + H2  
Na2CO3 + HCl = NaCl + H2O + CO2  
C7H6O2 + O2 = CO2 + H2O  
Fe2(SO4)3 + KOH = K2SO4 + Fe(OH)3  
Al2(SO4)3 + Ca(OH)2 = Al(OH)3 + CaSO4  
H2SO4 + HI = H2S + I2 + H2O  
Ca3(PO4)2 + SiO2 = P4O10 + CaSiO3  
Cu + HNO3 = Cu(NO3)2 + NO + H2O  
S + HNO3 = H2SO4 + NO2 + H2O  

All of the compounds should be separated by ' + ', and the reactants and products should be separated by ' = '. Each unbalanced equation is on a new line of the text file. The code assumes correct input of all chemical equations.

The Excel sheet PeriodicTableData.xlsx is necessary for any stoichiometric calculations.

## Helper File Along with any Imports

In [1]:
import xlrd
import tkinter as tk
import numpy as np
import sympy as sym
from tkinter.filedialog import askopenfile

def symbolAndMass (fileName):
    
    workbook = xlrd.open_workbook(fileName)
    sheet = workbook.sheet_by_index(0)
    symbolName = sheet.col_values(1, 1)
    mass = sheet.col_values(6, 1)

    symbolMassDict = {s : m for s, m in zip(symbolName, mass)}
    return (symbolMassDict)

def unParen(compound):
    import re 
    myRegEx = re.compile(r"(\()(\w*)(\))(\d*)",re.I)
    while (1): 
      myMatches = myRegEx.findall(compound)
      if (len(myMatches) == 0):
         return(compound)
      for match in myMatches:   
        text =""
        count = 1 if  match[3] == "" else int(match[3])
        text = match[1] * count
        compound = compound.replace('(' + match[1] + ')' + match[3], text)
        #print ("now", compound) 
     
def atomCount(compound):

  #return: dictionary - key is atom, value is # atoms

  #Modified from Lpez web resource ... stackoverflow.com/questions/16699180 ...
  import re

  atomDict = {}

  #unParentisize the given compound
  compoundNoParen = unParen(compound)

  myRegEx = re.compile("(C[laroudsemf]?|Os?|N[eaibdpos]?|S[icernbmg]?|P[drmtboau]?|H[eofgas]?|A[lrsgutcm]|B[eraik]?|Dy|E[urs]|F[erm]?|G[aed]|I[nr]?|Kr?|L[iaur]|M[gnodt]|R[buhenaf]|T[icebmalh]|U|V|W|Xe|Yb?|Z[nr])(\d*)")

  #create dictionary with key as atom, and value as # how many of this atom
  myMatches = myRegEx.findall(compoundNoParen)
  for match in myMatches:
    #Search atom
    atom = match[0]
    value = 1 if match[1]=="" else int(match[1])
    #print(atom,value)
    atomDict.setdefault(atom,0)
    atomDict [atom] += value

  #print (atomDict)
  return (atomDict)

def molesAndCompound (thisTerm):
    c = thisTerm[0]
    #print (c)
    if c.isdigit():
        noMoles = int (c)
        myCompound = thisTerm[1:]
    else:
        noMoles = 1
        myCompound = thisTerm
    return [noMoles,  myCompound ]

## Any functions that I made Myself

In [5]:
def makeform(window, reactants, products):
    global equations
    global loop_var
    entries = []
    unbalanced_reactants = []
    unbalanced_products = []
    column = 0
    equation=equations[loop_var]
    between_spaces = equation.split(' ')
    index = between_spaces.index('=')
    for i in range (0, index):
        unbalanced_reactants.append(between_spaces[i])
    for i in range (index+1, len(between_spaces)):
        unbalanced_products.append(between_spaces[i])    
    unbalanced_products[:] = [x for x in unbalanced_products if x != '+']
    unbalanced_reactants[:] = [x for x in unbalanced_reactants if x != '+']
    
    #display the unbalanced equation
    for reactant in unbalanced_reactants:
        if reactant != unbalanced_reactants[-1]:
            lab = tk.Label(window, text = reactant).grid(column=column, row=0)
            column = column+1
            lab2 = tk.Label(window, text=' + ').grid(column=column, row=0)
            column = column+1
        #the last reactant has to display either an '=' or '-->' after it, rather than '+'
        else:
            lab = tk.Label(window, text = reactant).grid(column=column, row=0)
            column = column+1
            lab2 = tk.Label(window, text=' --> ').grid(column=column, row=0)
            column = column+1
    for product in unbalanced_products:
        if product != unbalanced_products[-1]:
            lab = tk.Label(window, text = product).grid(column=column, row=0)
            column = column+1
            lab2 = tk.Label(window, text=' + ').grid(column=column, row=0)
            column = column+1
        #The last product has nothing displayed after it
        else:
            lab = tk.Label(window, text = product).grid(column=column, row=0)
    
    
    column = 0
    #for all of the reactants in the list, display them in the application window. Get their masses/moles
    for reactant in reactants:
        if reactant != reactants[-1]:
            lab = tk.Label(window, text = reactant).grid(column=column, row=1)
            ent = tk.Entry(window, width=10)
            ent.grid(column=column, row=2)
            column = column+1
            lab2 = tk.Label(window, text=' + ').grid(column=column, row=1)
            column = column+1
            entries.append((reactant, ent))
        #the last reactant has to display either an '=' or '-->' after it, rather than '+'
        else:
            lab = tk.Label(window, text = reactant).grid(column=column, row=1)
            ent = tk.Entry(window, width=10)
            ent.grid(column=column, row=2)
            column = column+1
            lab2 = tk.Label(window, text=' --> ').grid(column=column, row=1)
            column = column+1
            entries.append((reactant, ent))
    #Display all of the products in the application window and get any entries for their masses/moles
    for product in products:
        if product != products[-1]:
            lab = tk.Label(window, text = product).grid(column=column, row=1)
            ent = tk.Entry(window, width=10)
            ent.grid(column=column, row=2)
            column = column+1
            lab2 = tk.Label(window, text=' + ').grid(column=column, row=1)
            column = column+1
            entries.append((product, ent))
        #The last product has nothing displayed after it
        else:
            lab = tk.Label(window, text = product).grid(column=column, row=1)
            ent = tk.Entry(window, width=10)
            ent.grid(column=column, row=2) 
            entries.append((product, ent))
    return entries

#function to get the molar mass of a particular compound
def molar_mass(index, atom_counts, periodicTable):
    molar_mass = 0
    for key in atom_counts[index]:
        molar_mass = molar_mass + atom_counts[index][key] * float(periodicTable[key])
    return molar_mass

#calculate mass of a compound given the mass of another compound
def calculate_masses(molar_masses, texts, moles_and_compound):
    j = 0
    index = -2
    masses=[]
    for text in texts:
        if text != '':
            index=j
            break
        j=j+1
    for j in range(0,len(molar_masses)):
        if j == index:
            masses.append(float(texts[index]))
            continue
        masses.append(float(texts[index])/molar_masses[index]*moles_and_compound[j][0]*molar_masses[j]/moles_and_compound[index][0])
    return masses

#calculate the exact number of moles being produced in every other compound
def calculate_moles(molar_masses, texts, moles_and_compound):
    j=0
    index=-2 #not checking for any incorrect inputs so I guess this'll do
    moles=[]
    for text in texts:
        if text != '':
            index=j
            break
        j=j+1
    for j in range(0,len(molar_masses)):
        if j == index:
            moles.append(float(texts[index]))
            continue
        moles.append(float(texts[index])*moles_and_compound[j][0]/moles_and_compound[index][0])
    return moles

def fetch(entries, i):
    compounds = []
    texts = []
    for entry in entries:
        compound = entry[0]
        compounds.append(compound)
        text = entry[1].get()
        texts.append(text)
        
    periodicTable = symbolAndMass('PeriodicTableData.xlsx')
    moles_and_compound = []
    atom_counts = []
    molar_masses = []
    for compound in compounds:
        moles_and_compound.append(molesAndCompound(compound))
        atom_counts.append(atomCount(compound))
    for j in range(0, len(compounds)):
        molar_masses.append(molar_mass(j, atom_counts, periodicTable))
    masses = calculate_masses(molar_masses, texts, moles_and_compound)
    moles = calculate_moles(molar_masses, texts, moles_and_compound)
    if i.get() == 1:
        for k in range(0,len(molar_masses)):
            entries[k][1].delete(0,tk.END)
            entries[k][1].insert(0,str(moles[k]))
    elif i.get() == 0:
        for k in range(0,len(molar_masses)):
            entries[k][1].delete(0,tk.END)
            entries[k][1].insert(0,str(masses[k]))

#clear the contents of a tkinter window
def clearWindow(window):
    for widget in window.winfo_children():
        widget.destroy()

#destroy a tkinter window
def destroyWindow(window):
    window.destroy()
        
        
#function that allows user to choose a file to read
def chooseFile(window):
    file = askopenfile(initialdir="/")
    global equations
    global balanced_equations
    for line in file:
        equations.append(line.strip())
    window.destroy()
    balanced_equations = balanceEquations(equations)

#function to balance an array of chemical equations
#I didn't use any of the sympy symbols since I think that would be much harder to implement here
#the code used to balance the chemical equations was modified from a stack exchange post at
#https://stackoverflow.com/questions/45220032/how-to-balance-a-chemical-equation-in-python-2-7-using-matrices
#I tried implementing my own ideas using the sympy symbols method, but it would usually return an array of values with
#mostly zero, since that seems to be a trivial solution a lot of the time with any of the chemical equations
#I tried it on
def balanceEquations(equations):
    balanced_equations = []
    #making a for loop for each compound in the unbalanced chemical equation
    for equation in equations:
        
        #split the equation up into reactants and products without any "+" signs
        between_spaces = equation.split(' ')
        index = between_spaces.index('=')
        reactants = []
        products = []
        for i in range (0, index):
            reactants.append(between_spaces[i])
        for i in range (index+1, len(between_spaces)):
            products.append(between_spaces[i])    
        products[:] = [x for x in products if x != '+']
        reactants[:] = [x for x in reactants if x != '+']
        #getting a dictionary for all of the elements in the chemical equation
        #necessary to know how big to make the matrix
        left_compounds = [atomCount(compound) for compound in reactants]
        right_compounds = [atomCount(compound) for compound in products]
        elements = sorted(set().union(*left_compounds, *right_compounds))
        elements_index = dict(zip(elements, range(len(elements))))
        
        #make a matrix that's big enough to hold all of the coefficients in the system of linear equation
        w = len(left_compounds) + len(right_compounds)
        h = len(elements)
        A = [[0] * w for _ in range(h)]
        #load matrix with coefficients
        for col, compound in enumerate(left_compounds):
            for el, num in compound.items():
                row = elements_index[el]
                A[row][col] = num
        for col, compound in enumerate(right_compounds, len(left_compounds)):
            for el, num in compound.items():
                row = elements_index[el]
                A[row][col] = -num #right hand side gets multiplied by -1
        
        #solve the system
        #we solve for the nullspace of the matrix A, since in the equation Ax=b
        # x is the coefficients we need to solve for, and b is a zero vector (hence we need the nullspace)
        A = sym.Matrix(A)
        coeffs = A.nullspace()[0]
        #find least common denominator, multiply through to get integer solution
        coeffs *= sym.lcm([term.q for term in coeffs])
        
        balanced_reactants = " + ".join(["{}{}".format(coeffs[i], s) for i, s in enumerate(reactants)])
        balanced_products = " + ".join(["{}{}".format(coeffs[i], s) for i, s in enumerate(products, len(reactants))])
        balanced_equation = balanced_reactants + " = " + balanced_products
        balanced_equations.append(balanced_equation)
    return balanced_equations

#a function to get the next equation, actiavted by the button in the middle_window
def getNextEquation(middle_window, lab):
    global loop_var
    global reactants
    global products
    global equations
    global balanced_equations
    reactants.clear()
    products.clear()
    loop_var += 1
    if loop_var == len(balanced_equations):
        loop_var = 0
    equation=balanced_equations[loop_var]
    between_spaces = equation.split(' ')
    index = between_spaces.index('=')
    for i in range (0, index):
        reactants.append(between_spaces[i])
    for i in range (index+1, len(between_spaces)):
        products.append(between_spaces[i])    
    products[:] = [x for x in products if x != '+']
    reactants[:] = [x for x in reactants if x != '+']
    lab.config(text=equations[loop_var])
    

## Main Implementation

In [6]:
#main function, not implemented as a main function but as a separate cell

#create a window for the user to basically just click a button to choose a file
choice_window = tk.Tk()
choice_window.title("Stoichiometry")
choice_text = tk.Label(choice_window, text = "Choose a file of unbalanced chemical equations.")
choice_text.grid(column=0,row=0)
    
    
#User chooses a file of unbalanced chemical equations to read and store in the equations list
equations = []
balanced_equations = []
reactants = []
products = []
b = tk.Button(choice_window, text="Choose File",command=lambda win=choice_window:chooseFile(win))
b.grid(column=0,row=1)
#After the execution of the chooseFile function when the button is clicked
#the choice window will disappear, and a new window will appear giving the user
#A choice of which equation to balance and calculate mass/moles
choice_window.mainloop()


#Now the middle window pops up, which allows you to loop through all of the unbalanced equations in the
#file you choose, allowing you to pickone of them to balance (even though all of them are balanced by the program technically)
#but the one you choose is displayed in the third window that pops up
middle_window =tk.Tk()
middle_window.title("Stoichiometry")
lab = tk.Label(middle_window, text = equations[0])
lab.grid(column=0, row=0)
equation=balanced_equations[0]
between_spaces = equation.split(' ')
index = between_spaces.index('=')
for i in range (0, index):
    reactants.append(between_spaces[i])
for i in range (index+1, len(between_spaces)):
    products.append(between_spaces[i])    
products[:] = [x for x in products if x != '+']
reactants[:] = [x for x in reactants if x != '+']
loop_var = 0
but = tk.Button(middle_window, text = "Get Next Equation", command=lambda win=middle_window,lab=lab:getNextEquation(win, lab))
but.grid(column = 0, row=1)
quit_but = tk.Button(middle_window, text = "Choose This One", command=lambda win=middle_window:destroyWindow(win))
quit_but.grid(column=0, row=2)
middle_window.mainloop()


window = tk.Tk()
window.title("Stoichiometry")
#reads the file and populates the window with information about the chemical formula
ents = makeform(window, reactants, products) 
    
#Making the CheckButton to calculate moles
i=tk.IntVar() #boolean variable to check if the tickbox is checked or not
c1 = tk.Checkbutton(window, text="Moles", variable=i, onvalue=1, offvalue=0)
c1.grid(column=0, row=3)
b1 = tk.Button(window, text="Calculate",command=(lambda e=ents, boo=i: fetch(e,i)))
b1.grid(column=0,row=4)
    
window.mainloop()